#Movie suggestion using Content based recommendation system 

In [16]:
#import required packages
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


#Loading ratings data

In [17]:
pd.set_option('display.max_columns',100)
data = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')
data.sample(10)

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
113,114,Warrior,2011,PG-13,09 Sep 2011,140 min,"Action, Drama, Sport",Gavin O'Connor,"Gavin O'Connor (screenplay), Anthony Tambakis ...","Joel Edgerton, Tom Hardy, Nick Nolte, Jennifer...",The youngest son of an alcoholic former boxer ...,"English, Spanish",USA,Nominated for 1 Oscar. Another 6 wins & 20 nom...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.2/10,71.0,8.2,"358,261",tt1291584,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1212910-warrior/,20 Dec 2011,"$13,600,000",Lionsgate,http://www.WarriorFilm.com,True
93,94,Judgment at Nuremberg,1961,NOT RATED,18 Dec 1961,186 min,"Drama, War",Stanley Kramer,"Abby Mann, Abby Mann (based on his original st...","Spencer Tracy, Burt Lancaster, Richard Widmark...","In 1948, an American court in occupied Germany...","English, German",USA,Won 2 Oscars. Another 13 wins & 25 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.3/10,NaN,8.3,"49,303",tt0055031,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/judgment_at_nu...,07 Sep 2004,NaN,United Artists,NaN,True
226,227,Rope,1948,APPROVED,25 Sep 1948,80 min,"Crime, Drama, Thriller",Alfred Hitchcock,"Hume Cronyn (adapted by), Patrick Hamilton (fr...","John Dall, Farley Granger, Edith Evanson, Doug...","Two young men strangle their ""inferior"" classm...",English,USA,3 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.0/10,NaN,8.0,"102,340",tt0040746,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/rope/,06 Mar 2001,NaN,Warner Bros. Pictures,NaN,True
20,21,The Silence of the Lambs,1991,R,14 Feb 1991,118 min,"Crime, Drama, Thriller",Jonathan Demme,"Thomas Harris (novel), Ted Tally (screenplay)","Jodie Foster, Lawrence A. Bonney, Kasi Lemmons...",A young F.B.I. cadet must confide in an incarc...,English,USA,Won 5 Oscars. Another 49 wins & 37 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.6/10,84.0,8.6,"967,878",tt0102926,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/silence_of_the...,01 Jul 1998,NaN,Orion Pictures Corporation,NaN,True
44,45,Sunset Boulevard,1950,PASSED,29 Sep 1950,110 min,"Drama, Film-Noir",Billy Wilder,"Charles Brackett, Billy Wilder, D.M. Marshman Jr.","William Holden, Gloria Swanson, Erich von Stro...",A screenwriter is hired to rework a faded sile...,English,USA,Won 3 Oscars. Another 15 wins & 18 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.5/10,NaN,8.5,"151,983",tt0043014,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/sunset_boulevard/,26 Nov 2002,NaN,Paramount Pictures,NaN,True
77,78,Toy Story,1995,G,22 Nov 1995,81 min,"Animation, Adventure, Comedy",John Lasseter,"John Lasseter (original story by), Pete Docter...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney",A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 23 wins & 18 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.3/10,95.0,8.3,"680,986",tt0114709,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/toy_story/,20 Mar 2001,NaN,Buena Vista,http://www.disney.com/ToyStory,True
129,130,The Sixth Sense,1999,PG-13,06 Aug 1999,107 min,"Drama, Mystery, Thriller",M. Night Shyamalan,M. Night Shyamalan,"Bruce Willis, Haley Joel Osment, Toni Collette...",A boy who communicates with spirits that don't...,"English, Latin, Spanish",USA,Nominated for 6 Oscars. Another 3

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         250 non-null    int64  
 1   Title              250 non-null    object 
 2   Year               250 non-null    int64  
 3   Rated              250 non-null    object 
 4   Released           248 non-null    object 
 5   Runtime            250 non-null    object 
 6   Genre              250 non-null    object 
 7   Director           250 non-null    object 
 8   Writer             249 non-null    object 
 9   Actors             250 non-null    object 
 10  Plot               250 non-null    object 
 11  Language           250 non-null    object 
 12  Country            250 non-null    object 
 13  Awards             245 non-null    object 
 14  Poster             250 non-null    object 
 15  Ratings.Source     250 non-null    object 
 16  Ratings.Value      250 non

Contains 38 columns and 250 rows in the data.

In [19]:
data.shape

(250, 38)

Need only 4 columns for implementing the model. title, genre, directors, actors, plot

In [43]:
moviedata = pd.DataFrame(data[['Title','Genre','Director','Actors','Plot']])
moviedata

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...
...,...,...,...,...,...
245,The Lost Weekend,"Drama, Film-Noir",Billy Wilder,"Ray Milland, Jane Wyman, Phillip Terry, Howard...",The desperate life of a chronic alcoholic is f...
246,Short Term 12,Drama,Destin Daniel Cretton,"Brie Larson, John Gallagher Jr., Stephanie Bea...",A 20-something supervising staff member of a r...
247,His Girl Friday,"Comedy, Drama, Romance",Howard Hawks,"Cary Grant, Rosalind Russell, Ralph Bellamy, G...",A newspaper editor uses every trick in the boo...
248,The Straight Story,"Biography, Drama",David Lynch,"Sissy Spacek, Jane Galloway Heitz, Joseph A. C...",An old man makes a long journey by lawn-mover ...


Taking only 3 actors from the Actors coulmn.

In [22]:
moviedata['Actors'] = moviedata['Actors'].map(lambda x: x.split(',')[:3])


In [23]:
moviedata['Actors']

0            [Tim Robbins,  Morgan Freeman,  Bob Gunton]
1               [Marlon Brando,  Al Pacino,  James Caan]
2             [Al Pacino,  Robert Duvall,  Diane Keaton]
3        [Christian Bale,  Heath Ledger,  Aaron Eckhart]
4           [Martin Balsam,  John Fiedler,  Lee J. Cobb]
                             ...                        
245           [Ray Milland,  Jane Wyman,  Phillip Terry]
246    [Brie Larson,  John Gallagher Jr.,  Stephanie ...
247      [Cary Grant,  Rosalind Russell,  Ralph Bellamy]
248    [Sissy Spacek,  Jane Galloway Heitz,  Joseph A...
249           [Dev Patel,  Saurabh Shukla,  Anil Kapoor]
Name: Actors, Length: 250, dtype: object

Genre column has multiple genres combined together, 
making them into list of words and convertin them to lowercase.

In [24]:
moviedata['Genre']= moviedata['Genre'].map(lambda x: x.lower().split(','))
moviedata['Genre']

0                 [Crime,  Drama]
1                 [Crime,  Drama]
2                 [Crime,  Drama]
3        [Action,  Crime,  Drama]
4                 [Crime,  Drama]
                  ...            
245           [Drama,  Film-Noir]
246                       [Drama]
247    [Comedy,  Drama,  Romance]
248           [Biography,  Drama]
249                       [Drama]
Name: Genre, Length: 250, dtype: object

In [25]:
moviedata['Director'] = moviedata['Director'].map(lambda x: x.split(' '))
moviedata['Director']

0                      [Frank, Darabont]
1               [Francis, Ford, Coppola]
2               [Francis, Ford, Coppola]
3                   [Christopher, Nolan]
4                        [Sidney, Lumet]
                     ...                
245                      [Billy, Wilder]
246            [Destin, Daniel, Cretton]
247                      [Howard, Hawks]
248                       [David, Lynch]
249    [Danny, Boyle,, Loveleen, Tandan]
Name: Director, Length: 250, dtype: object

Will change all the words in the columns to lowercase.

In [26]:
for index, row in moviedata.iterrows():
  row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
  row['Director'] = ''.join(row['Director']).lower()


In [27]:
moviedata['Actors']

0                 [timrobbins, morganfreeman, bobgunton]
1                    [marlonbrando, alpacino, jamescaan]
2                  [alpacino, robertduvall, dianekeaton]
3             [christianbale, heathledger, aaroneckhart]
4                 [martinbalsam, johnfiedler, leej.cobb]
                             ...                        
245                [raymilland, janewyman, phillipterry]
246     [brielarson, johngallagherjr., stephaniebeatriz]
247           [carygrant, rosalindrussell, ralphbellamy]
248    [sissyspacek, janegallowayheitz, josepha.carpe...
249                [devpatel, saurabhshukla, anilkapoor]
Name: Actors, Length: 250, dtype: object

In [28]:
moviedata['Director']

0                  frankdarabont
1             francisfordcoppola
2             francisfordcoppola
3               christophernolan
4                    sidneylumet
                 ...            
245                  billywilder
246          destindanielcretton
247                  howardhawks
248                   davidlynch
249    dannyboyle,loveleentandan
Name: Director, Length: 250, dtype: object

In [29]:
#!pip install rake_nltk
from rake_nltk import Rake

#using rake to get key_words

In [31]:
# initializing the new column
moviedata['Key_words'] = ""

for index, row in moviedata.iterrows():
    plot = row['Plot']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
moviedata.drop(columns = ['Plot'], inplace = True)

In [32]:
moviedata

,Title,Genre,Director,Actors,key_words,Key_words
0,The Shawshank Redemption,"[Crime, Drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]",,"[common, decency, acts, years, two, imprisoned..."
1,The Godfather,"[Crime, Drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]",,"[organized, crime, dynasty, transfers, control..."
2,The Godfather: Part II,"[Crime, Drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]",,"[career, 1920s, new, york, early, life, vito, ..."
3,The Dark Knight,"[Action, Crime, Drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]",,"[people, greatest, psychological, wreaks, havo..."
4,12 Angry Men,"[Crime, Drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]",,"[miscarriage, colleagues, reconsider, justice,..."
...,...,...,...,...,...,...
245,The Lost Weekend,"[Drama, Film-Noir]",billywilder,"[raymilland, janewyman, phillipterry]",,"[day, drinking, bout, desperate, life, followe..."
246,Short Term 12,[Drama],destindanielcretton,"[brielarson, johngallagherjr., stephaniebeatriz]",,"[something, supervising, staff, member, co, wo..."
247,His Girl Friday,"[Comedy, Drama, Romance]",howardhawks,"[carygrant, rosalindrussell, ralphbellamy]",,"[keep, remarrying, book, wife, ace, reporter, ..."
248,The Straight Story,"[Biography, Drama]",davidlynch,"[sissyspacek, janegallowayheitz, josepha.carpe...",,"[relationship, mover, tractor, ill, brother, m..."


In [33]:
moviedata.set_index('Title', inplace = True)
moviedata

,Genre,Director,Actors,key_words,Key_words
Title,,,,,
The Shawshank Redemption,"[Crime, Drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]",,"[common, decency, acts, years, two, imprisoned..."
The Godfather,"[Crime, Drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]",,"[organized, crime, dynasty, transfers, control..."
The Godfather: Part II,"[Crime, Drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]",,"[career, 1920s, new, york, early, life, vito, ..."
The Dark Knight,"[Action, Crime, Drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]",,"[people, greatest, psychological, wreaks, havo..."
12 Angry Men,"[Crime, Drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]",,"[miscarriage, colleagues, reconsider, justice,..."
...,...,...,...,...,...
The Lost Weekend,"[Drama, Film-Noir]",billywilder,"[raymilland, janewyman, phillipterry]",,"[day, drinking, bout, desperate, life, followe..."
Short Term 12,[Drama],destindanielcretton,"[brielarson, johngallagherjr., stephaniebeatriz]",,"[something, supervising, staff, member, co, wo..."
His Girl Friday,"[Comedy, Drama, Romance]",howardhawks,"[carygrant, rosalindrussell, ralphbellamy]",,"[keep, remarrying, book, wife, ace, reporter, ..."


In [34]:
moviedata['bag_of_words'] = ''
columns = moviedata.columns
for index, row in moviedata.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
moviedata.drop(columns = [col for col in moviedata.columns if col!= 'bag_of_words'], inplace = True)

In [35]:
count = CountVectorizer()
count_matrix = count.fit_transform(moviedata['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(moviedata.index)
indices[:5]

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [36]:
count_matrix 

<250x2961 sparse matrix of type '<class 'numpy.int64'>'
	with 5342 stored elements in Compressed Sparse Row format>

In [37]:
cm = count_matrix.todense()

In [38]:
print(count_matrix[0,:])

  (0, 584)	1
  (0, 768)	1
  (0, 1011)	1
  (0, 2678)	1
  (0, 1810)	1
  (0, 306)	1
  (0, 519)	1
  (0, 655)	1
  (0, 59)	1
  (0, 2950)	1
  (0, 2765)	1
  (0, 1269)	1
  (0, 1733)	1
  (0, 311)	1
  (0, 969)	1
  (0, 2481)	1
  (0, 1899)	1
  (0, 888)	1
  (0, 2174)	1


In [39]:
count.vocabulary_

{'crime': 584,
 'drama': 768,
 'frankdarabont': 1011,
 'timrobbins': 2678,
 'morganfreeman': 1810,
 'bobgunton': 306,
 'common': 519,
 'decency': 655,
 'acts': 59,
 'years': 2950,
 'two': 2765,
 'imprisoned': 1269,
 'men': 1733,
 'bond': 311,
 'finding': 969,
 'solace': 2481,
 'number': 1899,
 'eventual': 888,
 'redemption': 2174,
 'francisfordcoppola': 1007,
 'marlonbrando': 1671,
 'alpacino': 118,
 'jamescaan': 1355,
 'organized': 1932,
 'dynasty': 793,
 'transfers': 2730,
 'control': 553,
 'clandestine': 475,
 'empire': 846,
 'aging': 82,
 'patriarch': 1972,
 'reluctant': 2193,
 'son': 2492,
 'robertduvall': 2264,
 'dianekeaton': 710,
 'career': 391,
 '1920s': 8,
 'new': 1870,
 'york': 2953,
 'early': 795,
 'life': 1575,
 'vito': 2854,
 'corleone': 563,
 'portrayed': 2057,
 'expands': 908,
 'family': 934,
 'syndicate': 2608,
 'michael': 1750,
 'tightens': 2670,
 'grip': 1131,
 'action': 56,
 'christophernolan': 464,
 'christianbale': 458,
 'heathledger': 1179,
 'aaroneckhart': 31,
 

In [40]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.15789474, 0.13764944, ..., 0.05263158, 0.05263158,
        0.05564149],
       [0.15789474, 1.        , 0.36706517, ..., 0.05263158, 0.05263158,
        0.05564149],
       [0.13764944, 0.36706517, 1.        , ..., 0.04588315, 0.04588315,
        0.04850713],
       ...,
       [0.05263158, 0.05263158, 0.04588315, ..., 1.        , 0.05263158,
        0.05564149],
       [0.05263158, 0.05263158, 0.04588315, ..., 0.05263158, 1.        ,
        0.05564149],
       [0.05564149, 0.05564149, 0.04850713, ..., 0.05564149, 0.05564149,
        1.        ]])

In [41]:
# function that takes in movie title as input and returns the top 10 recommended movies
def movie_recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    print(top_10_indexes)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(moviedata.index)[i])
        
    return recommended_movies

In [42]:
movie_recommendations('Fargo')

[132, 34, 226, 1, 61, 2, 100, 15, 214, 125]


['No Country for Old Men',
 'The Departed',
 'Rope',
 'The Godfather',
 'Reservoir Dogs',
 'The Godfather: Part II',
 'On the Waterfront',
 'Goodfellas',
 'Arsenic and Old Lace',
 'The Big Lebowski']